In [1]:
import os
import numpy as np
import csv
import pandas as pd
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import GloVeFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import FastTextFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import USEFeatureGeneration
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import collections
from imblearn.over_sampling import SMOTE


In [2]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]

with open("./results/word-embeddings-features/performance_DT_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [4]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled

        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Asthma
Macro F1 score: 0.9233003679327207 and Micro F1 Score 0.9288461538461539
CAD
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
CAD
Macro F1 score: 0.6190656565656566 and Micro F1 Score 0.6428571428571429
CHF
(12, 494, 300) (12,) Counter({1.0: 12})
Macro F1 score: 1 and Micro F1 Score 1
Depression
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Depression
Macro F1 score: 0.871801393051393 and Micro F1 Score 0.8795454545454545
Diabetes
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Diabetes
Macro F1 score: 0.6557539682539681 and Micro F1 Score 0.6714285714285715
Gallstones
(66, 495, 300) (66,) Counter({0.0: 56, 1.0: 10})
Gallstones
Macro F1 score: 0.7889300976800977 and Micro F1 Score 0.8037878787878787
GERD
(56, 495, 300) (56,) Counter({0.0: 49, 1.0: 7})
GERD
Macro F1 score: 0.875428737928738 and Micro F1 Score 0.8877777777777778
Gout
(67, 495, 300) (67,) Counter({0.0: 62, 1.0: 5})
Gout
Macro F1 score: 0.9090906367454356 and

In [6]:
with open("./results/word-embeddings-features/performance_DT_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = GloVeFeatureGeneration(train_preprocessed_df, morbidity).glove_matrix_gen()
    X = np.average(X, axis=1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Asthma
Macro F1 score: 0.932603238265003 and Micro F1 Score 0.9371794871794872
CAD
CAD
Macro F1 score: 0.6564285714285714 and Micro F1 Score 0.6714285714285715
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Depression
Macro F1 score: 0.9560751748251748 and Micro F1 Score 0.9583333333333334
Diabetes
Diabetes
Macro F1 score: 0.6102777777777778 and Micro F1 Score 0.6285714285714284
Gallstones
Gallstones
Macro F1 score: 0.8686487123987124 and Micro F1 Score 0.875
GERD
GERD
Macro F1 score: 0.8225126262626261 and Micro F1 Score 0.8377777777777778
Gout
Gout
Macro F1 score: 0.9815243579949463 and Micro F1 Score 0.9833333333333332
Hypercholesterolemia
Hypercholesterolemia
Macro F1 score: 0.7921500721500722 and Micro F1 Score 0.8097222222222221
Hypertension
Hypertension
Macro F1 score: 0.8395412920412919 and Micro F1 Score 0.8597222222222222
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
OA
Macro F1 score: 0.9344232238349885 and Micro F1 Score 0.9401515151515151
Obe

In [7]:
with open("./results/word-embeddings-features/performance_DT_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FastTextFeatureGeneration(train_preprocessed_df, morbidity).fasttext_matrix_gen()
    X = np.average(X, axis=1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
    
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Asthma
Macro F1 score: 0.9144313203136732 and Micro F1 Score 0.9198717948717949
CAD
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
CAD
Macro F1 score: 0.6659920634920635 and Micro F1 Score 0.6857142857142857
CHF
(12, 494, 300) (12,) Counter({1.0: 12})
Macro F1 score: 1 and Micro F1 Score 1
Depression
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Depression
Macro F1 score: 0.7248955211455211 and Micro F1 Score 0.7325757575757577
Diabetes
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Diabetes
Macro F1 score: 0.5001334776334775 and Micro F1 Score 0.5428571428571428
Gallstones
(66, 495, 300) (66,) Counter({0.0: 56, 1.0: 10})
Gallstones
Macro F1 score: 0.8198578300784183 and Micro F1 Score 0.8295454545454545
GERD
(56, 495, 300) (56,) Counter({0.0: 49, 1.0: 7})
GERD
Macro F1 score: 0.8838167388167388 and Micro F1 Score 0.8977777777777778
Gout
(67, 495, 300) (67,) Counter({0.0: 62, 1.0: 5})
Gout
Macro F1 score: 0.7706428783634666 a

In [8]:
with open("./results/word-embeddings-features/performance_DT_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = USEFeatureGeneration(train_preprocessed_df, morbidity).use_matrix_gen()
    X = np.average(X, axis=1)
    X = X.reshape(-1, 1)

    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma


2023-05-04 14:21:34.640387: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(66, 512) (66,) Counter({0.0: 63, 1.0: 3})
Asthma
Macro F1 score: 0.8544348788466433 and Micro F1 Score 0.8583333333333332
CAD
(62, 512) (62,) Counter({0.0: 35, 1.0: 27})
CAD
Macro F1 score: 0.6007142857142856 and Micro F1 Score 0.6428571428571429
CHF
(12, 512) (12,) Counter({1.0: 12})
Macro F1 score: 1 and Micro F1 Score 1
Depression
(66, 512) (66,) Counter({0.0: 58, 1.0: 8})
Depression
Macro F1 score: 0.8204433066933067 and Micro F1 Score 0.8378787878787879
Diabetes
(63, 512) (63,) Counter({1.0: 35, 0.0: 28})
Diabetes
Macro F1 score: 0.6059054834054833 and Micro F1 Score 0.6428571428571428
Gallstones
(66, 512) (66,) Counter({0.0: 56, 1.0: 10})
Gallstones
Macro F1 score: 0.7033749583749583 and Micro F1 Score 0.7143939393939394
GERD
(56, 512) (56,) Counter({0.0: 49, 1.0: 7})
GERD
Macro F1 score: 0.7098187923187924 and Micro F1 Score 0.7244444444444446
Gout
(67, 512) (67,) Counter({0.0: 62, 1.0: 5})
Gout
Macro F1 score: 0.6628702437912963 and Micro F1 Score 0.6820512820512821
Hyperchole